In [2]:
# !pip install langchain-mcp-adapters
# !pip install mcp

## Using with LangGraph StateGraph

In [1]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.prebuilt import ToolNode, tools_condition
from langchain.chat_models import init_chat_model

c:\Users\aniln\miniconda3\envs\channel\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

# Check if OPENAI_API_KEY is already set
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"]:
    # Ask user for input
    openai_key = input("Enter your OpenAI API key: ").strip()
    os.environ["OPENAI_API_KEY"] = openai_key
else:
    print("OPENAI_API_KEY already set.")

In [3]:
model = init_chat_model("openai:gpt-4.1")

In [11]:
client = MultiServerMCPClient(
    {
        "math": {
            "command": "python",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": ["./examples/math_server.py"],
            "transport": "stdio",
        },
        "weather": {
            # make sure you start your weather server on port 8000
            "url": "http://localhost:8000/mcp",
            "transport": "streamable_http",
        }
    }
)

In [ ]:
tools = await client.get_tools()

In [ ]:
def call_model(state: MessagesState):
    response = model.bind_tools(tools).invoke(state["messages"])
    return {"messages": response}

In [ ]:
builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_node(ToolNode(tools))
builder.add_edge(START, "call_model")
builder.add_conditional_edges(
    "call_model",
    tools_condition,
)
builder.add_edge("tools", "call_model")
graph = builder.compile()

In [ ]:
math_response = await graph.ainvoke({"messages": "what's (3 + 5) x 12?"})

In [ ]:
weather_response = await graph.ainvoke({"messages": "what is the weather in nyc?"})